In [195]:
from nested_lookup import nested_lookup

In [196]:
from pdfminer.high_level import extract_text,extract_pages
from pdfminer.layout import LAParams, LTContainer, LTTextLine

text = extract_text("../data/pacific-rim-2013_1.pdf" ,LAParams())
print(text)

PACIFIC RIM

by

Travis Beacham

PREFACE: LEXICON

(A supplementary guide to pertinent terms and jargon...)

• Anteverse. the older, parent universe on the other side of the 

Interstice

• bruiser. (slang) Jaeger gunner

• BTB. brain-to-brain interface (see PONS or DRIFT)

• conn-pod. the Jaeger's detachable cockpit module; sometimes 

called a Conn

• COSDEC. Pacific Rim Combined Special Defense Corp

• drift. (slang) a joint vision shared by Jaeger crew members 

while directly linked via the Pons; triggered by subconscious 
stimuli, a generally involuntary but seldom debilitating 
phenomenon 

• drivesuit. the Jaeger/pilot interface suit designed to monitor 
vital signs and to translate nerve signals to piloting input

• ghost-drift. (slang) a kind of quasi-telepathic vision shared 
(by parties with a pre-existing Pons connection) without the 
aid of a hard neural interface; commonly triggered by stress, 
physical proximity, or coinciding REM sleep patterns

• headspace. (slang) v

In [384]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LAParams, LTContainer, LTTextLine

def extract_layout(pdf_path):
    # Create a layout parameters object
    laparams = LAParams()

    # Parse the layout information
    layout_objects = []
    page_height=0
    page_width=0
    for page_layout in extract_pages(pdf_path, laparams=laparams):
        for obj in page_layout:
            if isinstance(obj, LTContainer):
                for text_line in obj:
                    if isinstance(text_line, LTTextLine):
                        layout_objects.append(text_line)
  

    page_height,page_width=page_layout.height,page_layout.width
    return layout_objects,(page_height,page_width)

# Example usage
pdf_path = '../data/pacific-rim-2013_1.pdf'
layout_objects,page_dim = extract_layout(pdf_path)





In [429]:
import json
import math

In [430]:
type(text)

str

In [431]:
import re
def is_int_ext_line(line):
    line=line.strip()
    if line.startswith("INT.") or line.startswith("EXT."):
        return True
    else:
        return False
    

## Transition

In [486]:
import re

transitionEnum = [
    "FADE IN",
    "FADE OUT",
    "FADE TO",  # Add a comma here
    "FADE UP",
    "JUMP CUT",
    "MATCH CUT",
    "SMASH MATCH CUT",
    "MATCH DISSOLVE",
    "CUT",
    "DISSOLVE",
    "FLASH CUT",
    "FREEZE FRAME",
    "IRIS IN",
    "IRIS OUT",
    "WIPE TO",
    "CUT TO",
    "CONTINUED",
    "TO WHITE",
    "TO BLACK"
]

def checkTransition(line, transitionEnum=None):
    line = line.strip().replace("\n", "")

    transitions_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(transition) for transition in transitionEnum) + r')\b', re.IGNORECASE)
    return bool(re.search(transitions_pattern, line))


## Character

In [487]:
def all_words_uppercase(text):
    # Remove non-alphabetic characters and split the text into words
    text = text.strip().replace("\n", "")
    
    # Update the regular expression to allow for numbers in words
    words = re.findall(r'\b[\w\d]+\b', text)

    # Check if all words, including those with numbers, are in uppercase
    return all(word.upper() == word for word in words)



def movie_character_position(movie_dict=None,frequency_count={}):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="character":
            frequency_count[math.ceil(value['bbox'][0])]=frequency_count.get(value['bbox'][0],0)+1
    frequency_count = dict(sorted(frequency_count.items(), key=lambda item: item[1],reverse=True))
    return frequency_count
            

def moviecharacter_precision_increase(movie_dict=None,max_value=0):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="character":
            if math.floor(value['bbox'][0])<=max_value<=math.ceil(value['bbox'][0]):
                print(i,math.ceil(value['bbox'][0]))
                movie_dict[i]['metadata']="character"
            else:
                movie_dict[i]['metadata']="unknown"
    return movie_dict
            



## Actionlines

In [488]:

def actionline_position(movie_dict=None,frequency_count={}):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            frequency_count[math.ceil(value['bbox'][0])]=frequency_count.get(value['bbox'][0],0)+1
    frequency_count = dict(sorted(frequency_count.items(), key=lambda item: item[1],reverse=True))
    return frequency_count
            

def actionline_increase_precision(movie_dict=None,max_value=0):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            if math.floor(value['bbox'][0])<=max_value<=math.ceil(value['bbox'][0]):
                print(i,math.ceil(value['bbox'][0]))
                movie_dict[i]['metadata']="action_line"
            else:
                movie_dict[i]['metadata']="unknown"
    return movie_dict
            

## Diaglouge


In [489]:

def dialogue_position(movie_dict=None,frequency_count={}):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            frequency_count[math.ceil(value['bbox'][0])]=frequency_count.get(value['bbox'][0],0)+1
    frequency_count = dict(sorted(frequency_count.items(), key=lambda item: item[1],reverse=True))
    return frequency_count
            

def dialogue_increase_precision(movie_dict=None,max_value=0):
    for i in movie_dict.keys():
        value=movie_dict[i]
        if value['metadata']=="unknown":
            if math.floor(value['bbox'][0])<=max_value<=math.ceil(value['bbox'][0]):
                print(i,math.ceil(value['bbox'][0]))
                movie_dict[i]['metadata']="dialogue"
            else:
                movie_dict[i]['metadata']="unknown"
    return movie_dict
            

## Paranthesis

In [490]:
def is_whole_text_in_parentheses(text):
    text=text.strip().replace("\n","")
    pattern = r'^\(.+\)$'
    return bool(re.match(pattern, text))

## next line

In [538]:
def replace_patterns(text):
    pattern = r'\b\d+\.\n'
    return re.sub(pattern, '', text)


In [537]:
def replace_next_line_empty_string(text):
    text=text.split("number:")[1]

    text=replace_patterns(text)

    if len(text)==0:
        return True
    else:
        return False


In [493]:
movie_dict={}

In [494]:
def create_and_save_json(movie_dict, filename="pdfminer.json"):
    with open(filename, "w") as file:
        json.dump(movie_dict, file)

In [495]:
movie_dict={}

In [496]:
# Print layout information
for i, text_line in enumerate(layout_objects):
    # print(f"Text Line {i + 1}:")
    # print(f"   - Text: {text_line.get_text()}")
    # print(f"   - Position: ({text_line.x0}, {text_line.y0})")
    # print(f"   - Size: ({text_line.width}, {text_line.height})")
    # print("-------------------------")
    bbox=(text_line.x0,text_line.y0,text_line.width,text_line.height)
    sentence_text=text_line.get_text()
    if is_int_ext_line(sentence_text):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"scene_info","bbox":bbox}
    elif checkTransition(sentence_text,transitionEnum=transitionEnum) and all_words_uppercase(sentence_text):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"scene_transition","bbox":bbox}
    elif all_words_uppercase(sentence_text):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"character","bbox":bbox}
        print(sentence_text)
    elif is_whole_text_in_parentheses(sentence_text):
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"paranthesis","bbox":bbox}
    else:
        movie_dict[f"{i+1}:number:"+sentence_text]={"metadata":"unknown","bbox":bbox}


    

PACIFIC RIM

PREFACE: LEXICON

BLACKNESS.

VOICE (FILTERED)

RALEIGH

VOICE (FILTERED)

RALEIGH

RALEIGH

2.

COUNSELOR

RALEIGH

COUNSELOR

RALEIGH

COUNSELOR

COUNSELOR (CONT’D)

RALEIGH

CNN ANCHOR (FILTERED)

3.

CNN ANCHOR (FILTERED) (CONT’D)

PENTECOST

PENTECOST (CONT’D)

MAKO

PENTECOST

4.

PENTECOST (CONT’D)

PENTECOST (CONT’D)

MAKO

FLICK (PRE-LAP)

JIM (FILTERED)

FLICK

CPO -- ?

JIM (FILTERED)

5.

FLICK

JIM (FILTERED)

FLICK

JIM (FILTERED)

FLICK

JIM (FILTERED)

FLICK

JIM (FILTERED)

FLICK

JIM (FILTERED)

FLICK

JIM (FILTERED)

FLICK

FLICK (CONT’D)

6.

FLICK

FLICK (CONT’D)

RALEIGH

FLICK

RALEIGH

FLICK

RALEIGH

FLICK

RALEIGH

FLICK

FLICK (CONT’D)

(MORE)

7.

FLICK (CONT’D)

RALEIGH

FLICK

RALEIGH

FLICK

RALEIGH

FLICK

FLICK (CONT’D)

RALEIGH

FLICK

RALEIGH

8.

FLICK

RALEIGH

FLICK

RALEIGH

RALEIGH (CONT’D)

9.

CALHOUN

CALHOUN

RALEIGH

CALHOUN

RALEIGH

CALHOUN

RALEIGH

10.

CALHOUN

RALEIGH

CALHOUN

RALEIGH

CALHOUN

RALEIGH

CALHOUN

CALHOUN (

NEWT

FLICK

NEWT

FLICK

29.

NEWT

FLICK

NEWT

FLICK

NEWT

FLICK

NEWT

NEWT (CONT’D)

FLICK

NEWT

NEWT (CONT’D)

30.

FLICK

NEWT

31.

FLICK

FLICK (CONT’D)

FLICK (CONT’D)

NEWT

FLICK

32.

FLICK (CONT’D)

NEWT

33.

NEWT

FLICK

FLICK (CONT’D)

34.

ANCHOR (O.S.)

FLICK

NEWT

FLICK

RALEIGH

FLICK (FILTERED)

INTERCUT FLICK/RALEIGH

RALEIGH

FLICK

RALEIGH

FLICK

RALEIGH

FLICK

RALEIGH

35.

FLICK

RALEIGH

RALEIGH (FILTERED)

PENTECOST (FILTERED)

RALEIGH

36.

PENTECOST (FILTERED)

PENTECOST (FILTERED)

RALEIGH

RALEIGH (CONT’D)

37.

MAKO (V.O.)

AKI (V.O.)

MAKO (V.O.)

AKI (V.O.)

RALEIGH

MAKO

PENTECOST (FILTERED)

RALEIGH (FILTERED)

38.

PENTECOST (FILTERED)

RALEIGH

MAKO (FILTERED)

RALEIGH

PENTECOST (FILTERED)

RALEIGH

PENTECOST (FILTERED)

RALEIGH

RALEIGH (FILTERED)

RALEIGH (FILTERED) (CONT’D)

39.

MAKO (V.O.)

RALEIGH

YANCY (V.O.)

RALEIGH

YOUNG MAKO

RALEIGH

YOUNG MAKO

40.

RALEIGH

YOUNG MAKO

YOUNG MAKO (CONT’D)

41.

PENTECOST

BENVENISTE

TAKADA

In [497]:
frequency_count=movie_character_position(movie_dict=movie_dict)
movie_dict=moviecharacter_precision_increase(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

frequency_count=actionline_position(movie_dict=movie_dict)
movie_dict=actionline_increase_precision(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

frequency_count=movie_character_position(movie_dict=movie_dict)
movie_dict=moviecharacter_precision_increase(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])

frequency_count=dialogue_position(movie_dict=movie_dict)
movie_dict=dialogue_increase_precision(movie_dict=movie_dict,max_value=list(frequency_count.keys())[0])



59:number:VOICE (FILTERED)
 252
68:number:RALEIGH
 252
70:number:VOICE (FILTERED)
 252
80:number:RALEIGH
 252
90:number:RALEIGH
 252
96:number:COUNSELOR
 252
100:number:RALEIGH
 252
104:number:COUNSELOR
 252
107:number:RALEIGH
 252
110:number:COUNSELOR
 252
115:number:COUNSELOR (CONT’D)
 252
121:number:RALEIGH
 252
132:number:CNN ANCHOR (FILTERED)
 252
146:number:CNN ANCHOR (FILTERED) (CONT’D)
 253
168:number:PENTECOST
 253
173:number:PENTECOST (CONT’D)
 253
175:number:MAKO
 253
177:number:PENTECOST
 253
182:number:PENTECOST (CONT’D)
 252
186:number:PENTECOST (CONT’D)
 252
190:number:MAKO
 252
199:number:FLICK (PRE-LAP)
 252
209:number:JIM (FILTERED)
 252
213:number:FLICK
 252
218:number:JIM (FILTERED)
 252
224:number:FLICK
 252
226:number:JIM (FILTERED)
 252
229:number:FLICK
 252
231:number:JIM (FILTERED)
 252
234:number:FLICK
 252
236:number:JIM (FILTERED)
 252
238:number:FLICK
 252
240:number:JIM (FILTERED)
 252
243:number:FLICK
 252
248:number:JIM (FILTERED)
 252
250:number:FLICK
 

In [539]:
def page_number_tag(movie_dict=None):
    for i in movie_dict.keys():
        value=movie_dict[i]
        
        if value['metadata']=="unknown":
            
            if replace_next_line_empty_string(i):
                movie_dict[i]['metadata']="page_number"
                print(i)
    return movie_dict

In [540]:
movie_dict=page_number_tag(movie_dict=movie_dict)

nik
95:number:2.

nik
145:number:3.

nik
180:number:4.

nik
223:number:5.

nik
267:number:6.

nik
307:number:7.

nik
352:number:8.

nik
396:number:9.

nik
444:number:10.

nik
495:number:11.

nik
529:number:12.

nik
577:number:13.

nik
620:number:14.

nik
660:number:15.

nik
703:number:16.

nik
750:number:17.

nik
790:number:18.

nik
832:number:19.

nik
880:number:20.

nik
920:number:21.

nik
966:number:22.

nik
1005:number:23.

nik
1043:number:24.

nik
1085:number:25.

nik
1133:number:26.

nik
1175:number:27.

nik
1223:number:28.

nik
1262:number:29.

nik
1310:number:30.

nik
1353:number:31.

nik
1396:number:32.

nik
1434:number:33.

nik
1479:number:34.

nik
1522:number:35.

nik
1573:number:36.

nik
1608:number:37.

nik
1646:number:38.

nik
1692:number:39.

nik
1730:number:40.

nik
1772:number:41.

nik
1812:number:42.

nik
1858:number:43.

nik
1902:number:44.

nik
1946:number:45.

nik
1990:number:46.

nik
2036:number:47.

nik
2081:number:48.

nik
2118:number:49.

nik
2165:number:50.

n

In [536]:
text="5401:number:sadgs"
text=text.split("number:")[1]
text

'sadgs'

In [519]:
create_and_save_json(movie_dict=movie_dict)

In [482]:
page_dim

(792, 612)

In [405]:
import re

# Example usage
result = all_words_uppercase("JiT 2\n")
print(result)  # Output: True


False


In [415]:
replace_patterns("75.\n")

'.\n'